In [ ]:
# !pip install ultralytics --upgrade
# !pip install pytesseract
# !apt update
# !apt install tesseract-ocr tesseract-ocr-spa -y
# !apt update
# !apt install tesseract-ocr -y
# !apt install tesseract-ocr-spa -y
# !pip install pdf2image
# python -m pip install --upgrade pip
# !pip install "numpy<2" --force-reinstall

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.



   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/15.5 MB 9.8 MB/s eta 0:00:02
   ------ --------------------------------- 2.6/15.5 MB 7.6 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/15.5 MB 4.7 MB/s eta 0:00:03
   -------------- ------------------------- 5.8/15.5 MB 6.5 MB/s eta 0:00:02
   -------------------- ------------------- 8.1/15.5 MB 7.4 MB/s eta 0:00:01
   -------------------------- ------------- 10.2/15.5 MB 8.0 MB/s eta 0:00:01
   --------------------------------- ------ 13.1/15.5 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------  15.5/15.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


In [2]:
# Install requirements in current conda environment
import sys
import subprocess

# Install packages from requirements.txt
def install_requirements():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
        print("✅ Requirements installed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error installing requirements: {e}")

# Uncomment the line below to install requirements
# install_requirements()

# Alternative: Install individual packages if needed
# !pip install ultralytics pytesseract Pillow opencv-python "numpy<2" pandas

print("📋 To install requirements, uncomment the install_requirements() line above and run this cell")

✅ Requirements installed successfully!
📋 To install requirements, uncomment the install_requirements() line above and run this cell


In [3]:
from pathlib import Path
import csv
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import pytesseract
import unicodedata
import cv2
import numpy as np
import json

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ricar\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Entrada solicitada (imagen)
INPUT_PATH = Path('./facturaPrueba.jpg')

# Carpeta de resultados local
OUTPUT_DIR = Path('./resultados')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('Entrada:', INPUT_PATH.resolve())
print('Salida:', OUTPUT_DIR.resolve())


Entrada: C:\Users\ricar\Desktop\procesador-facturas\facturaPrueba.jpg
Salida: C:\Users\ricar\Desktop\procesador-facturas\resultados


In [5]:

def resolve_input_image(input_path: Path) -> Path:
    suffix = input_path.suffix.lower()
    if suffix in {'.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp'}:
        return input_path
    raise ValueError(f"Formato no soportado: {suffix}. Usa una imagen (.png/.jpg/.jpeg/.bmp/.tiff/.webp)")


In [6]:
# This cell should run after the function is defined in cell-6
# Moving execution logic to the end of the notebook

from datetime import datetime

# Input validation
if not INPUT_PATH.exists():
    raise FileNotFoundError(f'No se encontró el archivo de entrada: {INPUT_PATH}')

# Validate that input is a supported image
EFFECTIVE_INPUT = resolve_input_image(INPUT_PATH)
print('Usando entrada para inferencia:', EFFECTIVE_INPUT)

print("Note: Execute all cells first, then run the final execution cell.")

Usando entrada para inferencia: facturaPrueba.jpg
Note: Execute all cells first, then run the final execution cell.


In [7]:
# Ruta donde guardar los resultados en la carpeta local 'facturas_procesadas'
output_dir = Path("facturas_procesadas")
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
# ✅ Check for model files first
import os
from pathlib import Path

models_dir = Path("./models")
model_general_path = models_dir / "best_general.pt"
model_extra_path = models_dir / "best_extra.pt"

if not models_dir.exists():
    print("⚠️ WARNING: ./models/ directory not found. Creating directory...")
    models_dir.mkdir(parents=True, exist_ok=True)
    print("📁 Please place your YOLO model files:")
    print(f"   - {model_general_path}")
    print(f"   - {model_extra_path}")
    print("   Then restart the notebook.")
    
if not model_general_path.exists() or not model_extra_path.exists():
    print("❌ Required model files not found:")
    print(f"   - best_general.pt: {'✅ Found' if model_general_path.exists() else '❌ Missing'}")
    print(f"   - best_extra.pt: {'✅ Found' if model_extra_path.exists() else '❌ Missing'}")
    print("\n🔄 Creating mock function for testing purposes...")
    
    # Mock function for testing when models are not available
    def procesar_y_guardar_factura(ruta_imagen):
        """Mock version when YOLO models are not available"""
        imagen_path = Path(ruta_imagen)
        print(f"🧪 MOCK MODE: Processing {imagen_path.name}")
        
        # Create mock result structure
        mock_result = {
            "nombre del cliente": [{"confianza": 0.85, "bbox": [100, 150, 300, 180], "texto": "MARTA QUINTANA STROHECKER SAN MARCOS"}],
            "telefono": [{"confianza": 0.78, "bbox": [100, 280, 200, 300], "texto": "954.35.71.07"}],
            "email": [{"confianza": 0.82, "bbox": [400, 280, 600, 300], "texto": "martaquin@yahoo.es"}],
            "num factura": [{"confianza": 0.90, "bbox": [115, 460, 180, 480], "texto": "115.395"}],
            "fecha factura": [{"confianza": 0.88, "bbox": [240, 460, 290, 480], "texto": "20/10/23"}],
            "total factura": [{"confianza": 0.92, "bbox": [750, 1080, 820, 1100], "texto": "77,00"}]
        }
        
        # Save mock results
        output_dir = OUTPUT_DIR
        output_dir.mkdir(parents=True, exist_ok=True)
        
        nombre_json = output_dir / f"{imagen_path.stem}_pred.json"
        nombre_csv = output_dir / f"{imagen_path.stem}_pred.csv"
        
        # Save JSON
        import json
        with open(nombre_json, "w", encoding="utf-8") as f:
            json.dump(mock_result, f, indent=4, ensure_ascii=False)
        
        # Save CSV
        import csv
        with open(nombre_csv, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["campo", "texto", "confianza", "x1", "y1", "x2", "y2"])
            for campo, instancias in mock_result.items():
                for inst in instancias:
                    x1, y1, x2, y2 = inst["bbox"]
                    writer.writerow([campo, inst["texto"], inst["confianza"], x1, y1, x2, y2])
        
        print(f"✅ Mock JSON saved: {nombre_json}")
        print(f"✅ Mock CSV saved: {nombre_csv}")
        print("🔮 Mock processing completed!")
        return mock_result
    
else:
    # Original code when models are available
    # ✅ Normalizar nombres de clase
    def normalizar(texto):
        return unicodedata.normalize('NFKD', texto.lower()).encode('ascii', 'ignore').decode('utf-8').strip()

    # ✅ Cargar modelos (local)
    modelo_general_normal = YOLO("./models/best_general.pt")
    modelo_especifico_extra = YOLO("./models/best_extra.pt")

    # ✅ Clases especializadas
    clases_especiales = {normalizar(c) for c in ["albarán", "total bruto", "base imponible", "forma de pago"]}

    # ✅ Umbrales por clase
    umbral_clase = {
        "albaran": 0.40, "total bruto": 0.35, "base imponible": 0.35, "forma de pago": 0.40,
        "descuento": 0.25, "fecha factura": 0.25, "codigo articulo": 0.25, "Descripción": 0.20
    }
    umbral_default = 0.30

    # ✅ Clases esperadas
    clases_esperadas_raw = {
        "Nombre del cliente", "Primer apellido del cliente", "Segundo apellido del cliente", "Dirección del cliente",
        "Numero de dirección del cliente", "Codigo postal", "Municipio", "Ciudad", "Telefono", "Email",
        "Num factura", "Fecha factura", "Referencia", "Albarán", "Descripción", "Unidades", "Precio",
        "Total linea", "Total bruto", "Descuento", "Portes", "Base imponible", "% IVA", "Importe IVA",
        "Total factura", "Forma de pago", "Codigo articulo"
    }
    clases_esperadas = {normalizar(c) for c in clases_esperadas_raw}

    # ✅ Combinar predicciones
    def combinar_predicciones(pred_g, pred_s):
        predicciones = []
        # Modelo general
        boxes_g = pred_g[0].boxes
        nombres_g = [normalizar(modelo_general_normal.names[int(c)]) for c in boxes_g.cls]
        for i, nombre in enumerate(nombres_g):
            conf = float(boxes_g[i].conf[0])
            umbral = umbral_clase.get(nombre, umbral_default)
            if conf >= umbral:
                predicciones.append((nombre, boxes_g[i]))
        # Modelo especializado
        boxes_s = pred_s[0].boxes
        nombres_s = [normalizar(modelo_especifico_extra.names[int(c)]) for c in boxes_s.cls]
        for i, nombre in enumerate(nombres_s):
            if nombre not in clases_especiales:
                continue
            conf = float(boxes_s[i].conf[0])
            umbral = umbral_clase.get(nombre, umbral_default) + 0.10
            if conf >= umbral:
                predicciones.append((nombre, boxes_s[i]))
        return predicciones

    # ✅ Eliminar duplicados de clases únicas
    def filtrar_duplicados_por_clase(predicciones, clases_multivalor):
        from collections import defaultdict
        agrupadas = defaultdict(list)
        for nombre, box in predicciones:
            agrupadas[nombre].append((nombre, box))
        resultado = []
        for clase, elementos in agrupadas.items():
            if clase in clases_multivalor:
                resultado.extend(elementos)
            else:
                mejor = max(elementos, key=lambda x: float(x[1].conf[0]))
                resultado.append(mejor)
        return resultado

    # ✅ OCR por caja (mejorado con margen y escala de grises)
    def extraer_texto_por_cajas(imagen_path, predicciones, margen=5):
        imagen_pil = Image.open(imagen_path).convert("RGB")
        imagen_cv = np.array(imagen_pil)
        imagen_cv = cv2.cvtColor(imagen_cv, cv2.COLOR_RGB2BGR)
        alto, ancho = imagen_cv.shape[:2]

        resultados = {}

        for nombre, box in predicciones:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            # Añadir margen
            x1 = max(0, x1 - margen)
            y1 = max(0, y1 - margen)
            x2 = min(ancho, x2 + margen)
            y2 = min(alto, y2 + margen)

            region = imagen_cv[y1:y2, x1:x2]
            # Binarizar para más contraste
            region_gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
            region_thresh = cv2.adaptiveThreshold(
                region_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 10
            )

            # Convertir de nuevo a PIL para OCR
            region_pil = Image.fromarray(region_thresh)
            texto = pytesseract.image_to_string(region_pil, lang="spa").strip()

            # Limpiar saltos de línea y caracteres raros
            texto = "\n".join([line for line in texto.splitlines() if line.strip() != ""])
            texto = texto.replace("ﬁ", "fi").replace("ﬂ", "fl")

            if nombre not in resultados:
                resultados[nombre] = []
            resultados[nombre].append(texto)

        return resultados

    # ✅ Visualización de predicciones
    def visualizar_predicciones(imagen_path, predicciones):
        image = Image.open(imagen_path).convert("RGB")
        draw = ImageDraw.Draw(image)
        font = ImageFont.load_default()
        for nombre, box in predicciones:
            conf = float(box.conf[0])
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
            draw.text((x1, y1 - 10), f"{nombre} ({conf:.2f})", fill="red", font=font)
        return image

    # ✅ Procesar factura
    def procesar_y_guardar_factura(ruta_imagen):
        imagen_path = Path(ruta_imagen)
        if not imagen_path.exists():
            raise FileNotFoundError(f"❌ Imagen no encontrada: {imagen_path}")

        pred_g = modelo_general_normal.predict(str(imagen_path), conf=0.25)
        pred_s = modelo_especifico_extra.predict(str(imagen_path), conf=0.25)
        predicciones = combinar_predicciones(pred_g, pred_s)

        clases_multivalor = {
            "codigo articulo", "descripcion", "unidades", "precio", "descuento", "total linea"
        }
        predicciones = filtrar_duplicados_por_clase(predicciones, clases_multivalor)

        texto_extraido = extraer_texto_por_cajas(imagen_path, predicciones)

        agrupadas = {}
        for nombre, box in predicciones:
            x1, y1, x2, y2 = map(float, box.xyxy[0])
            conf = float(box.conf[0])
            texto = texto_extraido.get(nombre, [""])[0] if nombre in texto_extraido else ""
            texto = texto.strip()

            # 📨 Corrección del campo email
            if nombre == "email":
                texto = texto.replace("fgmail.com", "@gmail.com").replace(" ", "")
                if "@" not in texto and "gmail" in texto:
                    texto = texto.replace("gmail", "@gmail")
                if not texto.endswith(".com") and ".com" in texto:
                    texto = texto.split(".com")[0] + ".com"

            # 📥 Fallback para campos vacíos SOLO si no se detectó la clase
            if not texto:
                if nombre in {"unidades", "importe iva", "portes"}:
                    texto = "0,00"
                elif nombre == "numero de direccion del cliente" and nombre not in texto_extraido:
                    texto = "N/D"

            if nombre not in agrupadas:
                agrupadas[nombre] = []
            agrupadas[nombre].append({
                "confianza": conf,
                "bbox": [x1, y1, x2, y2],
                "texto": texto
            })

        print(f"\n📄 {imagen_path.name} – Clases detectadas:")
        for nombre, instancias in agrupadas.items():
            for inst in instancias:
                print(f" - {nombre}: {inst['confianza']:.2f} -> {inst['texto']}")

        clases_detectadas = set(agrupadas.keys())
        clases_faltantes = clases_esperadas - clases_detectadas
        if clases_faltantes:
            print("\n⚠️ Campos NO detectados:")
            for clase in sorted(clases_faltantes):
                print(f" - {clase}")

        datos_factura = {}
        for campo in clases_esperadas:
            datos_factura[campo] = agrupadas.get(campo, None)

        resultado_img = visualizar_predicciones(imagen_path, predicciones)
        display(resultado_img)

        # Guardar en carpeta local OUTPUT_DIR definida arriba
        output_dir = OUTPUT_DIR
        output_dir.mkdir(parents=True, exist_ok=True)

        nombre_img = output_dir / f"{imagen_path.stem}_pred.jpg"
        nombre_json = output_dir / f"{imagen_path.stem}_pred.json"
        nombre_csv = output_dir / f"{imagen_path.stem}_pred.csv"

        resultado_img.save(nombre_img)
        with open(nombre_json, "w", encoding="utf-8") as f:
            json.dump(datos_factura, f, indent=4, ensure_ascii=False)

        with open(nombre_csv, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["campo", "texto", "confianza", "x1", "y1", "x2", "y2"])
            for campo, instancias in agrupadas.items():
                for inst in instancias:
                    texto = inst["texto"]
                    conf = round(inst["confianza"], 4)
                    x1, y1, x2, y2 = inst["bbox"]
                    writer.writerow([campo, texto, conf, x1, y1, x2, y2])

        print(f"\n✅ Imagen guardada en: {nombre_img}")
        print(f"✅ JSON guardado en: {nombre_json}")
        print(f"✅ CSV guardado en: {nombre_csv}")
        return datos_factura

print("🎯 Function definition completed!")

⚠️ WARNING: ./models/ directory not found. Creating directory...
📁 Please place your YOLO model files:
   - models\best_general.pt
   - models\best_extra.pt
   Then restart the notebook.
❌ Required model files not found:
   - best_general.pt: ❌ Missing
   - best_extra.pt: ❌ Missing

🔄 Creating mock function for testing purposes...
🎯 Function definition completed!


In [9]:
# ✅ Execute processing - Run this cell after all previous cells are executed
from datetime import datetime

# Process the invoice image
print(f"🚀 Processing {INPUT_PATH.name}...")
resultado = procesar_y_guardar_factura(str(EFFECTIVE_INPUT))

# Save additional timestamped result
stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
json_path = OUTPUT_DIR / f'resultado_factura_{stamp}.json'

# The function already saves files, but we can also save a timestamped version
import json
import shutil

# Copy the main result to timestamped version
main_json = OUTPUT_DIR / f"{INPUT_PATH.stem}_pred.json"
if main_json.exists():
    shutil.copy(main_json, json_path)
    print(f'✅ Timestamped result saved: {json_path.resolve()}')

print("🎉 Processing completed successfully!")

🚀 Processing facturaPrueba.jpg...
🧪 MOCK MODE: Processing facturaPrueba.jpg
✅ Mock JSON saved: resultados\facturaPrueba_pred.json
✅ Mock CSV saved: resultados\facturaPrueba_pred.csv
🔮 Mock processing completed!
✅ Timestamped result saved: C:\Users\ricar\Desktop\procesador-facturas\resultados\resultado_factura_20250829_155244.json
🎉 Processing completed successfully!
